In [2]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import numpy as np
import matplotlib.pyplot as plt

def generate_julia_set(c, x_min, x_max, y_min, y_max, max_iter,width, height):
    x = np.linspace(x_min, x_max, width)
    y = np.linspace(y_min, y_max, height)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y
    # Initialize the Julia set array
    julia = np.zeros(Z.shape, dtype=int)
    mask = np.ones(Z.shape, dtype=bool)

    for n in range(max_iter):
        Z[mask] = c * Z[mask] ** 3 + Z[mask] **2 + c
        mask = np.abs(Z) < 1000
        julia += mask

    return julia

def generate_mandelbrot_set(lmbda, x_min, x_max, y_min, y_max, max_iter, width, height):
    x = np.linspace(x_min, x_max, width)
    y = np.linspace(y_min, y_max, height)
    X, Y = np.meshgrid(x, y)
    C = X + 1j * Y

    mandelbrot = np.zeros(C.shape, dtype=int)
    Z = np.zeros(C.shape, dtype=complex)
    for n in range(max_iter):
        mask = np.abs(Z) <= 2
        Z[mask] = C[mask] * Z[mask]**3 + C[mask] * Z[mask]**2 + C[mask]
        mandelbrot[mask] = n - np.log2(np.log(np.abs(Z[mask])))

    mandelbrot = np.nan_to_num(mandelbrot)
    mandelbrot = mandelbrot / mandelbrot.max()
    return mandelbrot

def newton_julia_set(lmbda, x_min, x_max, y_min, y_max, max_iter, width, height):
    x = np.linspace(x_min, x_max, width)
    y = np.linspace(y_min, y_max, height)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y

    julia = np.zeros(Z.shape, dtype=int)
    roots = []

    for n in range(max_iter):
        old_Z = Z.copy()
        Z -= (lmbda * Z**3 + Z**2 + lmbda) / (3 * lmbda * Z**2 + 2 * Z)
        for i in np.unique(Z):
            if abs(i - old_Z).max() < 1e-6 and i not in roots:
                roots.append(i)
        julia += np.abs(Z - old_Z) < 1e-6

    return julia, roots

def on_generate_newton_julia_click():
    try:
        lmbda = complex(entry_lambda.get())
        x_min, x_max = float(entry_xmin.get()), float(entry_xmax.get())
        y_min, y_max = float(entry_ymin.get()), float(entry_ymax.get())
        max_iter = int(entry_max_iter.get())

        newton_julia, roots = newton_julia_set(lmbda, x_min, x_max, y_min, y_max, max_iter, width=400, height=400)
        
        # Plotting the Julia set
        plt.imshow(newton_julia, extent=[x_min, x_max, y_min, y_max], cmap='hot')
        # Plotting the roots
        for root in roots:
            plt.plot(root.real, root.imag, 'go')  # Mark roots with green dots

        plt.colorbar()
        plt.title(f"Newton Julia Set for λ = {lmbda} with roots")
        plt.xlabel("Real part")
        plt.ylabel("Imaginary part")
        plt.savefig("newton_julia_set_temp.png")
        plt.close()

        img = Image.open("newton_julia_set_temp.png")
        img = img.resize((400, 400))
        photo = ImageTk.PhotoImage(img)

        label_image.config(image=photo)
        label_image.image = photo
    except Exception as e:
        messagebox.showerror("Error", str(e))
    
def on_generate_click(generator_func):
    try:
        lmbda = complex(entry_lambda.get())
        x_min, x_max = float(entry_xmin.get()), float(entry_xmax.get())
        y_min, y_max = float(entry_ymin.get()), float(entry_ymax.get())
        max_iter = int(entry_max_iter.get())

        result_set = generator_func(lmbda, x_min, x_max, y_min, y_max, max_iter, width=400, height=400)
        
        plt.imshow(result_set, extent=[x_min, x_max, y_min, y_max], cmap='hot')
        plt.colorbar()
        plt.title(f"Set for λ = {lmbda}")
        plt.xlabel("Real part")
        plt.ylabel("Imaginary part")
        plt.savefig("result_set_temp.png")
        plt.close()

        img = Image.open("result_set_temp.png")
        img = img.resize((400, 400))
        photo = ImageTk.PhotoImage(img)

        label_image.config(image=photo)
        label_image.image = photo
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Tkinter window setup
root = tk.Tk()
root.title("Set Generator")

# Create input fields and labels for parameters
label_lambda = tk.Label(root, text="λ (complex):")
label_lambda.pack()
entry_lambda = tk.Entry(root)
entry_lambda.pack()

label_xmin = tk.Label(root, text="x_min:")
label_xmin.pack()
entry_xmin = tk.Entry(root)
entry_xmin.pack()

label_xmax = tk.Label(root, text="x_max:")
label_xmax.pack()
entry_xmax = tk.Entry(root)
entry_xmax.pack()

label_ymin = tk.Label(root, text="y_min:")
label_ymin.pack()
entry_ymin = tk.Entry(root)
entry_ymin.pack()

label_ymax = tk.Label(root, text="y_max:")
label_ymax.pack()
entry_ymax = tk.Entry(root)
entry_ymax.pack()

label_max_iter = tk.Label(root, text="Max Iterations:")
label_max_iter.pack()
entry_max_iter = tk.Entry(root)
entry_max_iter.pack()

# Add buttons for generating Julia and Mandelbrot sets
button_generate_julia = tk.Button(root, text="Generate Julia Set", command=lambda: on_generate_click(generate_julia_set))
button_generate_julia.pack()
button_generate_mandelbrot = tk.Button(root, text="Generate Mandelbrot Set", command=lambda: on_generate_click(generate_mandelbrot_set))
button_generate_mandelbrot.pack()
button_generate_newton_julia = tk.Button(root, text="Generate Newton Julia Set", command=on_generate_newton_julia_click)
button_generate_newton_julia.pack()

# Label for displaying the image
label_image = tk.Label(root)
label_image.pack()

# Run the Tkinter loop
root.mainloop()
